In [18]:
import pandas as pd

In [19]:
heatmapData = pd.read_csv('heatmapDataWithState.csv')

In [20]:
MISSING_VALUE = 131
heatmapData["hasMissing"] = (heatmapData["mean_evi"] > MISSING_VALUE)

In [21]:
heatmapData['mean_evi'] = heatmapData['mean_evi']/131
heatmapData = heatmapData.sort_values(["x", "y", "year"])

heatmapData

,year,x,y,mean_evi,state,hasMissing
132529,2001,8,26,0.549618,WA,False
341901,2004,8,26,0.312977,WA,False
551232,2007,8,26,0.442748,WA,False
1109592,2015,8,26,0.580153,WA,False
1388805,2019,8,26,0.496183,WA,False
...,...,...,...,...,...,...
1417391,2020,478,65,0.572519,ME,False
1487210,2021,478,65,0.641221,ME,False
1557032,2022,478,65,0.755725,ME,False
1626829,2023,478,65,0.717557,ME,False


In [22]:
heatmapData.mean_evi.describe()

count    1.744939e+06
mean     5.765990e-01
std      2.027674e-01
min      2.290076e-02
25%      4.274809e-01
50%      6.030534e-01
75%      7.557252e-01
max      9.694656e-01
Name: mean_evi, dtype: float64

In [23]:
DiffHeatmapData = heatmapData.copy()
DiffHeatmapData["year_diff"] = DiffHeatmapData.groupby(["x", "y"])["mean_evi"].diff()
DiffHeatmapData[DiffHeatmapData.year_diff.isna()].year.unique() # Due to mistmatch and first group!

array([2001, 2000, 2014, 2004, 2007, 2005, 2003, 2002, 2009, 2021, 2017,
       2015, 2008, 2019, 2010, 2012, 2016, 2006, 2024, 2013, 2022, 2011,
       2018])

In [24]:
DiffHeatmapData = DiffHeatmapData.dropna(subset=['year_diff'])

In [25]:
agg = DiffHeatmapData.groupby(['x', 'y', 'state'])['year_diff'] \
                     .agg(['std', 'count']) \
                     .reset_index()
agg

,x,y,state,std,count
0,8,26,WA,0.180791,4
1,9,25,WA,0.029941,24
2,9,26,WA,0.036504,24
3,9,27,WA,0.030863,24
4,9,28,WA,0.047106,24
...,...,...,...,...,...
69976,477,63,ME,0.029261,24
69977,477,64,ME,0.100790,24
69978,477,65,ME,0.038465,24
69979,477,66,ME,0.029261,24


In [26]:
og = pd.read_csv('heatmap_data.csv')
reference = og[['x', 'y', 'value']]
reference['hasMissing'] = reference['value'] > 131
result = reference.merge(agg, on = ['x', 'y'], how='left') 
result

C:\Users\Ricky\AppData\Local\Temp\ipykernel_16932\3411903896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference['hasMissing'] = reference['value'] > 131


,x,y,value,hasMissing,state,std,count
0,0,0,104,False,NaN,NaN,NaN
1,1,0,102,False,NaN,NaN,NaN
2,2,0,78,False,NaN,NaN,NaN
3,3,0,99,False,NaN,NaN,NaN
4,4,0,104,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...
262139,507,255,133,True,NaN,NaN,NaN
262140,508,255,133,True,NaN,NaN,NaN
262141,509,255,133,True,NaN,NaN,NaN
262142,510,255,133,True,NaN,NaN,NaN


In [27]:
result.to_csv("heatmapSd.csv", index=False)

In [12]:
result['count'].sort_values()

247021    1.0
14074     1.0
242508    1.0
34829     1.0
241941    1.0
         ... 
262139    NaN
262140    NaN
262141    NaN
262142    NaN
262143    NaN
Name: count, Length: 262144, dtype: float64

In [13]:
result['state'].unique()

array([nan, 'MN', 'WA', 'ID', 'MT', 'ND', 'MI', 'ME', 'WI', 'OR', 'SD',
       'NH', 'WY', 'NY', 'VT', 'IA', 'NE', 'MA', 'IL', 'PA', 'RI', 'CA',
       'NV', 'UT', 'OH', 'CT', 'IN', 'NJ', 'CO', 'WV', 'MO', 'KS', 'DE',
       'MD', 'VA', 'KY', 'NM', 'OK', 'AZ', 'TN', 'TX', 'AR', 'NC', 'SC',
       'MS', 'AL', 'GA', 'LA', 'FL'], dtype=object)

In [14]:
print(result['hasMissing'].value_counts())

hasMissing
False    184384
True      77760
Name: count, dtype: int64


In [16]:
print(result['mean'].isna().sum())


122182


In [16]:
result[result['std'].isna()].head()

,x,y,value,hasMissing,state,std,count
0,0,0,104,False,NaN,NaN,NaN
1,1,0,102,False,NaN,NaN,NaN
2,2,0,78,False,NaN,NaN,NaN
3,3,0,99,False,NaN,NaN,NaN
4,4,0,104,False,NaN,NaN,NaN


In [17]:
reference['value'].describe()

count    262144.000000
mean         94.833527
std          34.200965
min           3.000000
25%          70.000000
50%         100.000000
75%         133.000000
max         133.000000
Name: value, dtype: float64